## First  Task:

#### Libraries are will be imported:

In [231]:
# importing libraries
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
print('Libraries imported.')

Libraries imported.


#### At next we will download the website as text

In [232]:
# Getting the text of the webpage from the given wikipedia link.
raw_wikipedia_page= requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

# using beautiful as described in the task.
soup = BeautifulSoup(raw_wikipedia_page,'xml')
print('Data downloaded!')

Data downloaded!


In [233]:
#Lets have a look at the imported text:
#print(soup.prettify())

#### The command in the next lines extracts only the table form the text

In [ ]:
# extracting the table from the imported text
table = soup.find('table')
print("Table extracted")

#### Here we change the data as stated by the description of the first task. Detailed explainations can be found in the comments below:

In [235]:
#Definition of empty dataframes
Postcode      = []
Borough       = []
Neighbourhood = []

# Insert the table columns into the predefined dataframes
for tr_cell in table.find_all('tr'):
    
    # We define dummy variables which will be filled in the for loop.
    counter = 1
    Postcode_var      = 'DUMMY'
    Borough_var       = 'DUMMY'
    Neighbourhood_var = 'DUMMY'
    
    for td_cell in tr_cell.find_all('td'):
        if counter == 1: 
            Postcode_var = td_cell.text
        if counter == 2: 
            Borough_var = td_cell.text
        if counter == 3: 
            Neighbourhood_var = str(td_cell.text).strip()
            
        counter +=1
        
    #If borough is unassigned we skip the entry
    if (Borough_var.find('Not assigned')==0):
        continue
    #If for some reason the dummy variables still have their default values, we also skip the entry.
    if(Postcode_var.find('DUMMY')==0 or Borough_var.find('DUMMY')==0 or Neighbourhood_var.find('DUMMY')==0):
        continue
    #If neighbourhood is unassigned and borough is filled corretly we set Neighbourhood_var=Borough_var
    if (Neighbourhood_var.find('Not assigned')==0): 
        Neighbourhood_var=Borough_var
         
    Postcode.append(Postcode_var)
    Borough.append(Borough_var)
    Neighbourhood.append(Neighbourhood_var)

In [236]:
#Quick check if all arrays have the same length:
print('length postcode:', len(Postcode))
print('length borough:', len(Borough))
print('length neighbourhood:', len(Neighbourhood))

length postcode: 103
length borough: 103
length neighbourhood: 103


In [237]:
unique_p = set(Postcode)
print('number of unique entires in postcode:', len(unique_p)) #Postal codes on the wikipedia link are unique! There are no duplicates as stated in the description of the task!
unique_b = set(Borough)
print('number of unique entires in postcode:', len(unique_b))

number of unique entires in postcode: 103
number of unique entires in postcode: 10


In [238]:
Postcode_unique_list      = []
Borough_unique_list       = []
Neighbourhood_unique_list = []


for pc in unique_pc:
    p_dummy = ''; b_dummy = ''; n_dummy = ''; 
    for pc_array_idx, pc_array_val in enumerate(Postcode):
        if pc == pc_array_val:
            p_dummy = pc_array_val;
            b_dummy = Borough[pc_array_idx]
            if n_dummy == '': 
                n_dummy = Neighbourhood[pc_array_idx]
            else:
                # Here we concatinate different values of neighbourhood for the same value of borough.
                n_dummy = n_dummy + ', ' + Neighbourhood[pc_array_idx]
                
    Postcode_unique_list.append(p_dummy.rstrip('\n'))
    Borough_unique_list.append(b_dummy.rstrip('\n'))
    Neighbourhood_unique_list.append(n_dummy.rstrip('\n'))

#### Result of the first task:

In [239]:
toronto_dict = {'Postcode':Postcode_unique_list, 'Borough':Borough_unique_list, 'Neighbourhood':Neighbourhood_unique_list}
df_toronto = pd.DataFrame.from_dict(toronto_dict)
df_toronto.head()

,Postcode,Borough,Neighbourhood
0,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest..."
1,M5B,Downtown Toronto,"Garden District, Ryerson"
2,M5M,North York,"Bedford Park, Lawrence Manor East"
3,M4G,East York,Leaside
4,M4J,East York,"East Toronto, Broadview North (Old East York)"


In [251]:
df_toronto.shape

(103, 3)

## Second Task:

#### Load of latitudes and longitudes from the csv

In [252]:
lat_lon = pd.read_csv('https://cocl.us/Geospatial_data')
lat_lon.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Column "Postal Code" needs to be renamed for the merge

In [254]:
lat_lon.rename(columns={'Postal Code':'Postcode'},inplace=True)
lat_lon.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Result of the second task:

In [256]:
df_toronto_lat_lon = pd.merge(df_toronto,lat_lon,on='Postcode')
df_toronto_lat_lon.head(5)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,M5M,North York,"Bedford Park, Lawrence Manor East",43.733283,-79.419750
3,M4G,East York,Leaside,43.709060,-79.363452
4,M4J,East York,"East Toronto, Broadview North (Old East York)",43.685347,-79.338106


## Third Task:

#### Getting all the boroughs that have "Toronto" in their name

In [274]:
df_subset = df_toronto_lat_lon[df_toronto_lat_lon['Borough'].str.contains('Toronto',regex=False)]
df_subset.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
5,M6P,West Toronto,"High Park, The Junction South",43.661608,-79.464763
7,M6J,West Toronto,"Little Portugal, Trinity",43.647927,-79.419750
9,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572


#### Libraries are will be imported:

In [264]:
import requests # library to handle requests

!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

In [265]:
# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


! pip install folium==0.5.0
import folium # plotting library

     |████████████████████████████████| 79 kB 8.5 MB/s  eta 0:00:01
  Created wheel for folium: filename=folium-0.5.0-py3-none-any.whl size=76240 sha256=dd1cb89be02b2e7e1337e5c04907857e8c29476d276fd44368c05f94d367a0f5
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/b2/2f/2c/109e446b990d663ea5ce9b078b5e7c1a9c45cca91f377080f8
Successfully built folium


#### Coordinates are marked on a map of Toronto:

In [270]:
map_toronto = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

for lat,lng,borough,neighbourhood in zip(df_subset['Latitude'],df_subset['Longitude'],df_subset['Borough'],df_subset['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)
map_toronto

#### Clustering algorithm is performed in the next step:

In [269]:
k=5
toronto_clustering = df_subset.drop(['Postcode','Borough','Neighbourhood'],1)
kmeans = KMeans(n_clusters = k,random_state=0).fit(toronto_clustering)
kmeans.labels_
df_subset.insert(0, 'Cluster Labels', kmeans.labels_)

In [272]:
df_subset.head()

,Cluster Labels,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,4,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049
1,0,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
5,3,M6P,West Toronto,"High Park, The Junction South",43.661608,-79.464763
7,1,M6J,West Toronto,"Little Portugal, Trinity",43.647927,-79.419750
9,2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572


#### Postcodes are marked according to their clustering:

In [273]:
# create map
map_clusters = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, neighbourhood, cluster in zip(df_subset['Latitude'], df_subset['Longitude'], df_subset['Neighbourhood'], df_subset['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters